In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
API_KEY=os.getenv("GOOGLE_API_KEY")

In [5]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0.2,
    google_api_key=API_KEY
)

In [6]:
llm

ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), temperature=0.2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002B47C1A5B80>, default_metadata=(), model_kwargs={})

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import LLMChain
from langchain_classic.chains import SequentialChain
import PyPDF2

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
        "a": "choice here",
        "b": "choice here",
        "c": "choice here",
        "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
        "a": "choice here",
        "b": "choice here",
        "c": "choice here",
        "d": "choice here"
        },
        "correct": "correct answer"
    }
}

In [9]:
TEMPLATE="""
Text:{text}

You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs

### RESPONSE_JSON
{response_json}

"""

In [10]:
quiz_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [11]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=quiz_prompt,
    output_key="quiz",
    verbose=True
)

C:\Users\Harsh_Kr\AppData\Local\Temp\ipykernel_21028\3773648120.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(


In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity \
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student \
Quiz_MCQs: 
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_eval_prompt = PromptTemplate(
    input_variables=["quiz","subject"],
    template=TEMPLATE2
)

In [14]:
review_chain = LLMChain(
    llm=llm,
    prompt=quiz_eval_prompt,
    output_key="review",
    verbose=True
)

In [15]:
generate_evaluate_chain= SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","review"], 
    verbose=True
)

In [16]:
file_path=r'C:\Users\Harsh_Kr\OneDrive\Desktop\GenAI\mcqgen\data.txt'

In [17]:
with open(file_path, 'r') as file:
    text = file.read()

In [18]:
text

'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[3][4]\n\nFrom a theoretical viewpoint, probably approximate

In [19]:
NUMBER=5
SUBJECT="Machine Learning"
TONE="Simple"

In [20]:
response = generate_evaluate_chain(
    {
        "text": text,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)

C:\Users\Harsh_Kr\AppData\Local\Temp\ipykernel_21028\576823411.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  response = generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory dat

In [21]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[3][4]\n\nFrom a theoretical viewpoint, probably ap

In [22]:
quiz=response.get('quiz')

In [23]:
quiz

'```json\n{\n  "1": {\n    "mcq": "What is the primary goal of Machine Learning (ML) as described in the text?",\n    "options": {\n      "a": "To provide explicit instructions for every task.",\n      "b": "To develop statistical algorithms that learn from data and generalize to unseen data.",\n      "c": "To focus solely on natural language processing.",\n      "d": "To create computer games with pre-programmed rules."\n    },\n    "correct": "b"\n  },\n  "2": {\n    "mcq": "Who coined the term \'machine learning\' in 1959?",\n    "options": {\n      "a": "Donald Hebb",\n      "b": "Tom M. Mitchell",\n      "c": "Arthur Samuel",\n      "d": "Ian Goodfellow"\n    },\n    "correct": "c"\n  },\n  "3": {\n    "mcq": "According to the text, how does Deep Learning fit within the broader fields of Machine Learning and Artificial Intelligence?",\n    "options": {\n      "a": "Deep Learning is a separate field from both ML and AI.",\n      "b": "Deep Learning is a subset of Machine Learning, 

In [24]:
import re

def jsonParser(quiz):
    
    clean_output = quiz.strip("'")
    match = re.search(r"```json\s*(.*?)\s*```", clean_output, re.DOTALL)

    if match:
        json_string = match.group(1).strip()
        
        try:
            data = json.loads(json_string)
            print("JSON loaded successfully into a Python dictionary.")
            return data
            
        except json.JSONDecodeError as e:
            print(f"Error: Failed to decode the cleaned JSON string: {e}")
            print("--- Problematic String ---")
            print(json_string)
            return json_string
            
    else:
        print("Error: Could not find '```json' or the closing '```' in the output.")

In [25]:
json_string=jsonParser(quiz)

JSON loaded successfully into a Python dictionary.


In [26]:
json_string

{'1': {'mcq': 'What is the primary goal of Machine Learning (ML) as described in the text?',
  'options': {'a': 'To provide explicit instructions for every task.',
   'b': 'To develop statistical algorithms that learn from data and generalize to unseen data.',
   'c': 'To focus solely on natural language processing.',
   'd': 'To create computer games with pre-programmed rules.'},
  'correct': 'b'},
 '2': {'mcq': "Who coined the term 'machine learning' in 1959?",
  'options': {'a': 'Donald Hebb',
   'b': 'Tom M. Mitchell',
   'c': 'Arthur Samuel',
   'd': 'Ian Goodfellow'},
  'correct': 'c'},
 '3': {'mcq': 'According to the text, how does Deep Learning fit within the broader fields of Machine Learning and Artificial Intelligence?',
  'options': {'a': 'Deep Learning is a separate field from both ML and AI.',
   'b': 'Deep Learning is a subset of Machine Learning, which is a subset of Artificial Intelligence.',
   'c': 'Artificial Intelligence is a subset of Deep Learning.',
   'd': 'Mac